# Keras 训练中使用TensorBoard

Keras在回调函数中内置Tensorboard函数：

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc0


In [2]:
tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None
)

* **Scalars** 显示了如何将loss与每个时间段改变。还可以使用它来跟踪训练速度，学习率和其他标量值。
* **Graphs** 进行了可视化模型。在这种情况下，将现实层的Keras图，确保模型构建
* **Distributions**和**Histograms**：显示张量随时间的分布。这对于可视化权重和偏差并验证它们是否以预期的方式变化很有用

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
print(tf.__version__)
print(np.__version__)

2.2.0-rc0
1.18.1


In [4]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
mnist = np.load("mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'], mnist['y_train'], mnist['x_test'], mnist['y_test']
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [7]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [8]:
model = MyModel()

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="keras_logv1",
                                                     histogram_freq=1,
                                                     profile_batch=100000000)

model.fit(x=x_train,
         y=y_train,
         epochs=20,
         validation_data=(x_test, y_test),
         callbacks=[tensorboard_callback])

Epoch 1/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1374 - accuracy: 0.9591 - val_loss: 0.0587 - val_accuracy: 0.9798
Epoch 2/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0420 - accuracy: 0.9868 - val_loss: 0.0503 - val_accuracy: 0.9838
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0221 - accuracy: 0.9929 - val_loss: 0.0589 - val_accuracy: 0.9818
Epoch 4/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0135 - accuracy: 0.9954 - val_loss: 0.0594 - val_accuracy: 0.9833
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0087 - accuracy: 0.9971 - val_loss: 0.0824 - val_accuracy: 0.9797
Epoch 6/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0072 - accuracy: 0.9975 - val_loss: 0.0661 - val_accuracy: 0.9844
Epoch 7/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0057 - accuracy: 0.9982 - val_loss: 0.0734 - val_accuracy:

# 自定义模型

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# @tf.function
def train_step(images, labels):
    with tf.GradientTapea() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [10]:
model = MyModel()

In [ ]:
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
import os
logdir = os.path.join("logs/"+stamp)

summary_writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True, profiler=True)
EPOCHS=20

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
        train_step(x_train, y_train)
        
    with summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy,result(), step=epoch)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                      train_loss.result(),
                      train_accuracy.result() * 100,
                      test_loss.result(),
                      test_accuracy.result() * 100))
    
    # Reset metrics every epoch
    train_loss.reset_states_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=3, profiler_outdir=logdir)